In [ ]:
from keras.models import Sequential
from keras.layers import Dense
from sklearn.preprocessing import MinMaxScaler
from ARIMA import ARIMA_forecasts, ARIMA_forecast
from random_walk import random_walk_forecast
from helper_functions import compute_rmse, load_monthly_baci_data
import numpy as np

# Prepare the dataset
def create_ann_dataset(data, lookback, is_test=False, ):
    X, y = [], []
    if is_test:  # for test data, we just need the last entry for 1-step ahead forecast
        X.append(data[-lookback:])
        return np.array(X).reshape(1, lookback, 1), None
    else:
        for i in range(lookback, len(data)):
            X.append(data[i-lookback:i])
            y.append(data[i])
        return np.array(X), np.array(y)

# Design the ANN model
def create_ann_model(lookback):
    model = Sequential()
    model.add(Dense(8, input_dim=lookback, activation='relu'))
    model.add(Dense(1))
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

def ANN_forecast(train, test, lookback=2):
    # Rescale data
    scaler = MinMaxScaler(feature_range=(0, 1))
    train_scaled = scaler.fit_transform(train.values.reshape(-1, 1))
    test_scaled = scaler.transform(test.values.reshape(-1, 1))
    
    X_train, y_train = create_ann_dataset(train_scaled, lookback)
    
    model = create_ann_model(lookback)
    model.fit(X_train, y_train, epochs=50, batch_size=1)

    # Predict 1-step ahead
    X_test, _ = create_ann_dataset(np.concatenate((train_scaled, test_scaled), axis=0), lookback, is_test=True)
    forecast_scaled = model.predict(X_test)
    forecast = scaler.inverse_transform(forecast_scaled)
    
    return forecast.flatten()

def ARIMA_vs_RW_vs_ANN(data, train_ratio=0.8, months_ahead=1, n_forecasts=1):
    train_size = int(train_ratio * len(data))
    arima_rmses, rw_rmses, ann_rmses = [], [], []

    for _ in range(n_forecasts):
        train = data["CLOSE"].iloc[:train_size]
        test = data["CLOSE"].iloc[train_size:train_size + months_ahead]

        # ARIMA forecast
        arima_forecast = ARIMA_forecast(train, test)
        arima_rmse = compute_rmse(test, arima_forecast)
        arima_rmses.append(arima_rmse)

        # Random Walk forecast
        rw_forecast = random_walk_forecast(train, test)
        rw_rmse = compute_rmse(test, rw_forecast)
        rw_rmses.append(rw_rmse)

        # ANN forecast
        ann_forecast = ANN_forecast(train, test)
        ann_rmse = compute_rmse(test, ann_forecast)
        ann_rmses.append(ann_rmse)

        train_size += months_ahead

    return arima_rmses, rw_rmses, ann_rmses

df = load_monthly_baci_data()
arima_rmses, rw_rmses, ann_rmses = ARIMA_vs_RW_vs_ANN(df, n_forecasts=30)
print(f"ARIMA RMSEs: {np.average(arima_rmses)}")
print(f"Random Walk RMSEs: {np.average(rw_rmses)}")
print(f"ANN RMSEs: {np.average(ann_rmses)}")


/Users/mariusbolstad/.pyenv/versions/3.10.4/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/Users/mariusbolstad/.pyenv/versions/3.10.4/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/Users/mariusbolstad/.pyenv/versions/3.10.4/lib/python3.10/site-packages/statsmodels/tsa/base/tsa_model.py:473: ValueWarning: No frequency information was provided, so inferred frequency M will be used.
  self._init_dates(dates, freq)
/Users/mariusbolstad/.pyenv/versions/3.10.4/lib/python3.10/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "
/Users/mariusbolstad/VSCode/project-thes

Epoch 1/50
234/234 [==============================] - 1s 1ms/step - loss: 0.0253
Epoch 2/50
234/234 [==============================] - 0s 1ms/step - loss: 0.0141
Epoch 3/50
234/234 [==============================] - 0s 1ms/step - loss: 0.0089
Epoch 4/50
234/234 [==============================] - 0s 1ms/step - loss: 0.0062
Epoch 5/50
234/234 [==============================] - 0s 1ms/step - loss: 0.0053
Epoch 6/50
234/234 [==============================] - 0s 1ms/step - loss: 0.0053
Epoch 7/50
234/234 [==============================] - 0s 1ms/step - loss: 0.0049
Epoch 8/50
234/234 [==============================] - 0s 1ms/step - loss: 0.0047
Epoch 9/50
234/234 [==============================] - 0s 1ms/step - loss: 0.0046
Epoch 10/50
234/234 [==============================] - 0s 1ms/step - loss: 0.0044
Epoch 11/50
234/234 [==============================] - 0s 1ms/step - loss: 0.0044
Epoch 12/50
234/234 [==============================] - 0s 1ms/step - loss: 0.0043
Epoch 13/50
234/234 [====